In [1]:
# pip install fasttext
# pip install huggingface_hub

import fasttext
from huggingface_hub import hf_hub_download

# download model and get the model path
# cache_dir is the path to the folder where the downloaded model will be stored/cached.
model_path = hf_hub_download(repo_id="cis-lmu/glotlid", filename="model.bin", cache_dir=None)
print("model path:", model_path)

# load the model
model = fasttext.load_model(model_path)

model.bin:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

model path: /Users/nubster/.cache/huggingface/hub/models--cis-lmu--glotlid/snapshots/74cb50b709c9eefe0f790030c6c95c461b4e3b77/model.bin


In [1]:
%pip install fasttext

  Using cached fasttext-0.9.3.tar.gz (73 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-macosx_14_0_arm64.whl size=287404 sha256=a7ff114605bf00027d005f180d2448ccff2128e0e07fea02aa6f326f0bbb9b82
  Stored in directory: /Users/nubster/Library/Caches/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [3]:
len(model.labels)


2102

In [17]:
import pandas as pd
language_mapping = pd.read_csv('language_mapping.csv')
language_mapping_index = language_mapping.set_index('FLORES-200 code')['Language'].to_dict()
all_languages = list(language_mapping_index.keys())
from datasets import load_dataset
dataset = load_dataset("facebook/flores", "all")  # Loads all languages

In [7]:
len(all_languages)

204

In [12]:
non_existant = []
for lang in all_languages:
    if "__label__{}".format(lang) not in model.labels:
        non_existant.append(lang)

In [15]:
non_existant

['acq_Arab',
 'aka_Latn',
 'est_Latn',
 'grn_Latn',
 'kon_Latn',
 'pes_Arab',
 'prs_Arab',
 'tgl_Latn',
 'yue_Hant',
 'zho_Hans',
 'zho_Hant']

In [18]:
s = dataset['devtest'][0]["sentence_{}".format('acq_Arab')]

In [ ]:
redo_l={'aka_Latn':'twi_Latn','est_Latn':'ekk_Latn','grn_Latn':'gug_Latn','kon_Latn':'kng_Latn','tgl_Latn':'fil_Latn'}

In [27]:
model.predict(s)[0][0][9:]


'arb_Arab'

In [31]:
round(model.predict(s)[1][0],2)

0.57

In [32]:
languages_to_run = all_languages

In [34]:
from tqdm import tqdm
result_categories = {}
result_percentages = {}
result_accuracies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0
    
    result_categories[language_code]=[]
    result_percentages[language_code]=[]
    
    size = 1012
    for i in tqdm(range(size)): #length of devtest
        sentence = dataset['devtest'][i]["sentence_{}".format(language_code)]
        reply = model.predict(sentence)
        pred_lang = reply[0][0][9:]
        pred_percent = round(reply[1][0],2)
        result_categories[language_code].append(pred_lang)
        result_percentages[language_code].append(pred_percent)
        
        if pred_lang == language_code:
            accurate+=1
            
    result_accuracies[language_code] = accurate

    df = pd.DataFrame({
    "answer": result_categories[language_code],
    "percentage": result_percentages[language_code]
    })

    # Save to CSV
    df.to_csv("lid_predicted_GlotLID/{}.csv".format(language_code), index=False)

    print(accurate)
print(result_accuracies)

ace_Arab


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [00:01<00:00, 837.22it/s]


1003
ace_Latn


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [00:01<00:00, 762.23it/s]


1009
acm_Arab


...

0
zsm_Latn


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [00:01<00:00, 802.89it/s]


886
zul_Latn


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [00:01<00:00, 849.66it/s]

967
{'ace_Arab': 1003, 'ace_Latn': 1009, 'acm_Arab': 403, 'acq_Arab': 0, 'aeb_Arab': 526, 'afr_Latn': 1012, 'ajp_Arab': 227, 'aka_Latn': 0, 'als_Latn': 1012, 'amh_Ethi': 1012, 'apc_Arab': 346, 'arb_Arab': 879, 'arb_Latn': 1011, 'ars_Arab': 61, 'ary_Arab': 812, 'arz_Arab': 703, 'asm_Beng': 1012, 'ast_Latn': 1003, 'awa_Deva': 899, 'ayr_Latn': 1012, 'azb_Arab': 992, 'azj_Latn': 1012, 'bak_Cyrl': 1012, 'bam_Latn': 886, 'ban_Latn': 971, 'bel_Cyrl': 1012, 'bem_Latn': 1001, 'ben_Beng': 1011, 'bho_Deva': 988, 'bjn_Arab': 999, 'bjn_Latn': 990, 'bod_Tibt': 969, 'bos_Latn': 465, 'bug_Latn': 1005, 'bul_Cyrl': 1012, 'cat_Latn': 1012, 'ceb_Latn': 1002, 'ces_Latn': 1012, 'cjk_Latn': 972, 'ckb_Arab': 1004, 'crh_Latn': 994, 'cym_Latn': 1012, 'dan_Latn': 1007, 'deu_Latn': 1012, 'dik_Latn': 1001, 'dyu_Latn': 601, 'dzo_Tibt': 1009, 'ell_Grek': 1012, 'eng_Latn': 1012, 'epo_Latn': 1012, 'est_Latn': 0, 'eus_Latn': 1011, 'ewe_Latn': 1012, 'fao_Latn': 1012, 'fij_Latn': 1011, 'fin_Latn': 1012, 'fon_Latn': 1012,

In [1]:
redo_l={'aka_Latn':'twi_Latn','est_Latn':'ekk_Latn','grn_Latn':'gug_Latn','kon_Latn':'kng_Latn','tgl_Latn':'fil_Latn','pes_Arab':'fas_Arab','prs_Arab':'fas_Arab','zho_Hans':'cmn_Hani'}

In [3]:
result_accuracies= {'ace_Arab': 1003, 'ace_Latn': 1009, 'acm_Arab': 403, 'acq_Arab': 0, 'aeb_Arab': 526, 'afr_Latn': 1012, 'ajp_Arab': 227, 'aka_Latn': 0, 'als_Latn': 1012, 'amh_Ethi': 1012, 'apc_Arab': 346, 'arb_Arab': 879, 'arb_Latn': 1011, 'ars_Arab': 61, 'ary_Arab': 812, 'arz_Arab': 703, 'asm_Beng': 1012, 'ast_Latn': 1003, 'awa_Deva': 899, 'ayr_Latn': 1012, 'azb_Arab': 992, 'azj_Latn': 1012, 'bak_Cyrl': 1012, 'bam_Latn': 886, 'ban_Latn': 971, 'bel_Cyrl': 1012, 'bem_Latn': 1001, 'ben_Beng': 1011, 'bho_Deva': 988, 'bjn_Arab': 999, 'bjn_Latn': 990, 'bod_Tibt': 969, 'bos_Latn': 465, 'bug_Latn': 1005, 'bul_Cyrl': 1012, 'cat_Latn': 1012, 'ceb_Latn': 1002, 'ces_Latn': 1012, 'cjk_Latn': 972, 'ckb_Arab': 1004, 'crh_Latn': 994, 'cym_Latn': 1012, 'dan_Latn': 1007, 'deu_Latn': 1012, 'dik_Latn': 1001, 'dyu_Latn': 601, 'dzo_Tibt': 1009, 'ell_Grek': 1012, 'eng_Latn': 1012, 'epo_Latn': 1012, 'est_Latn': 0, 'eus_Latn': 1011, 'ewe_Latn': 1012, 'fao_Latn': 1012, 'fij_Latn': 1011, 'fin_Latn': 1012, 'fon_Latn': 1012, 'fra_Latn': 1012, 'fur_Latn': 1012, 'fuv_Latn': 998, 'gaz_Latn': 1012, 'gla_Latn': 1012, 'gle_Latn': 1012, 'glg_Latn': 1005, 'grn_Latn': 0, 'guj_Gujr': 1012, 'hat_Latn': 1012, 'hau_Latn': 1012, 'heb_Hebr': 1012, 'hin_Deva': 1009, 'hne_Deva': 960, 'hrv_Latn': 957, 'hun_Latn': 1012, 'hye_Armn': 1012, 'ibo_Latn': 1012, 'ilo_Latn': 1012, 'ind_Latn': 1009, 'isl_Latn': 1011, 'ita_Latn': 1011, 'jav_Latn': 1010, 'jpn_Jpan': 1012, 'kab_Latn': 1010, 'kac_Latn': 1012, 'kam_Latn': 969, 'kan_Knda': 1012, 'kas_Arab': 1008, 'kas_Deva': 1011, 'kat_Geor': 1012, 'kaz_Cyrl': 1011, 'kbp_Latn': 1011, 'kea_Latn': 956, 'khk_Cyrl': 1012, 'khm_Khmr': 1011, 'kik_Latn': 1009, 'kin_Latn': 962, 'kir_Cyrl': 1012, 'kmb_Latn': 1004, 'kmr_Latn': 998, 'knc_Arab': 970, 'knc_Latn': 1012, 'kon_Latn': 0, 'kor_Hang': 1012, 'lao_Laoo': 1011, 'lij_Latn': 1011, 'lim_Latn': 999, 'lin_Latn': 1011, 'lit_Latn': 1011, 'lmo_Latn': 1004, 'ltg_Latn': 1005, 'ltz_Latn': 1012, 'lua_Latn': 1005, 'lug_Latn': 1005, 'luo_Latn': 1011, 'lus_Latn': 1008, 'lvs_Latn': 1012, 'mag_Deva': 987, 'mai_Deva': 1009, 'mal_Mlym': 1012, 'mar_Deva': 1012, 'min_Arab': 994, 'min_Latn': 1006, 'mkd_Cyrl': 1011, 'mlt_Latn': 1012, 'mni_Beng': 1012, 'mos_Latn': 995, 'mri_Latn': 1010, 'mya_Mymr': 1012, 'nld_Latn': 1011, 'nno_Latn': 987, 'nob_Latn': 1004, 'npi_Deva': 1012, 'nso_Latn': 1010, 'nus_Latn': 1011, 'nya_Latn': 1011, 'oci_Latn': 1012, 'ory_Orya': 1012, 'pag_Latn': 1011, 'pan_Guru': 1012, 'pap_Latn': 1010, 'pbt_Arab': 1012, 'pes_Arab': 0, 'plt_Latn': 1010, 'pol_Latn': 1012, 'por_Latn': 1012, 'prs_Arab': 0, 'quy_Latn': 744, 'ron_Latn': 1011, 'run_Latn': 992, 'rus_Cyrl': 1012, 'sag_Latn': 1008, 'san_Deva': 1002, 'sat_Olck': 1012, 'scn_Latn': 1008, 'shn_Mymr': 1012, 'sin_Sinh': 1012, 'slk_Latn': 1011, 'slv_Latn': 1012, 'smo_Latn': 1010, 'sna_Latn': 1012, 'snd_Arab': 1012, 'som_Latn': 1012, 'sot_Latn': 999, 'spa_Latn': 1011, 'srd_Latn': 1010, 'srp_Cyrl': 1011, 'ssw_Latn': 1003, 'sun_Latn': 1008, 'swe_Latn': 1012, 'swh_Latn': 980, 'szl_Latn': 1001, 'tam_Taml': 1012, 'taq_Latn': 977, 'taq_Tfng': 1007, 'tat_Cyrl': 1011, 'tel_Telu': 1011, 'tgk_Cyrl': 1012, 'tgl_Latn': 0, 'tha_Thai': 1012, 'tir_Ethi': 976, 'tpi_Latn': 1011, 'tsn_Latn': 1008, 'tso_Latn': 1009, 'tuk_Latn': 1012, 'tum_Latn': 1010, 'tur_Latn': 1012, 'twi_Latn': 1011, 'tzm_Tfng': 77, 'uig_Arab': 1012, 'ukr_Cyrl': 1012, 'umb_Latn': 990, 'urd_Arab': 1011, 'uzn_Latn': 1012, 'vec_Latn': 1011, 'vie_Latn': 1012, 'war_Latn': 1012, 'wol_Latn': 1010, 'xho_Latn': 1004, 'ydd_Hebr': 1012, 'yor_Latn': 1012, 'yue_Hant': 0, 'zho_Hans': 0, 'zho_Hant': 0, 'zsm_Latn': 886, 'zul_Latn': 967}

In [2]:
non_exist = ['acq_Arab',
 'yue_Hant',
 'zho_Hant']

In [5]:
import os
import pandas as pd
for language_code in list(redo_l.keys()):
    correct_answer = redo_l[language_code]
    result_list = []
    
    reply_path = os.path.join('lid_predicted_GlotLID', "{}.csv".format(language_code))
    df = pd.read_csv(reply_path)
    acc = 0 
    for i in range(1012):
        reply = df['answer'][i]
        if reply == correct_answer:
            acc+=1
            
    result_accuracies[language_code]=acc
    print(acc)


1008
1011
1012
994
1011
1012
1012
994


In [ ]:
for key in non_exist:
    result_accuracies[key]=''

In [6]:
result_accuracies

{'ace_Arab': 1003,
 'ace_Latn': 1009,
 'acm_Arab': 403,
 'acq_Arab': 0,
 'aeb_Arab': 526,
 'afr_Latn': 1012,
 'ajp_Arab': 227,
 'aka_Latn': 1008,
 'als_Latn': 1012,
 'amh_Ethi': 1012,
 'apc_Arab': 346,
 'arb_Arab': 879,
 'arb_Latn': 1011,
 'ars_Arab': 61,
 'ary_Arab': 812,
 'arz_Arab': 703,
 'asm_Beng': 1012,
 'ast_Latn': 1003,
 'awa_Deva': 899,
 'ayr_Latn': 1012,
 'azb_Arab': 992,
 'azj_Latn': 1012,
 'bak_Cyrl': 1012,
 'bam_Latn': 886,
 'ban_Latn': 971,
 'bel_Cyrl': 1012,
 'bem_Latn': 1001,
 'ben_Beng': 1011,
 'bho_Deva': 988,
 'bjn_Arab': 999,
 'bjn_Latn': 990,
 'bod_Tibt': 969,
 'bos_Latn': 465,
 'bug_Latn': 1005,
 'bul_Cyrl': 1012,
 'cat_Latn': 1012,
 'ceb_Latn': 1002,
 'ces_Latn': 1012,
 'cjk_Latn': 972,
 'ckb_Arab': 1004,
 'crh_Latn': 994,
 'cym_Latn': 1012,
 'dan_Latn': 1007,
 'deu_Latn': 1012,
 'dik_Latn': 1001,
 'dyu_Latn': 601,
 'dzo_Tibt': 1009,
 'ell_Grek': 1012,
 'eng_Latn': 1012,
 'epo_Latn': 1012,
 'est_Latn': 1011,
 'eus_Latn': 1011,
 'ewe_Latn': 1012,
 'fao_Latn': 1012